In [35]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import re
import urllib.request
import requests
import time
from tqdm import tqdm

from collections import OrderedDict
import pymysql
import pymysql.cursors

In [36]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

# Drop Duplicate Brands

In [3]:
# setup information
config = {
   'host':'localhost',
   'port':3306, # default connection
   'user':'msba', # mysql default user name
   'password':'hku2019',
   'db':'makeup', # database 
   'charset':'utf8',
   'cursorclass':pymysql.cursors.DictCursor,
   }
 
    
# connect to database
con = pymysql.connect(**config)


# execute sql statement
try:
    with con.cursor() as cursor:
        sql = "SELECT brand_name from products GROUP BY brand_name"
        cursor.execute(sql)
        brands = cursor.fetchall() 
    
finally:
    con.close();

# convert into dataframe
df = pd.DataFrame(brands)
df.head()

,brand_name
0,canmake/井田
1,SUQQU
2,three
3,Paula‘s Choice/宝拉珍选
4,Laroche Posay/理肤泉


In [4]:
# 708 unique brands
split_name_lst = []

for idx in range(len(df)):
    brand_name = df.iloc[idx, 0]
    name_lst = brand_name.split('/')
    split_name_lst.append(name_lst)
    
df['brand_name_split'] = split_name_lst
df

,brand_name,brand_name_split
0,canmake/井田,"[canmake, 井田]"
1,SUQQU,[SUQQU]
2,three,[three]
3,Paula‘s Choice/宝拉珍选,"[Paula‘s Choice, 宝拉珍选]"
4,Laroche Posay/理肤泉,"[Laroche Posay, 理肤泉]"
5,Elizabeth Arden/雅顿,"[Elizabeth Arden, 雅顿]"
6,FANCL,[FANCL]
7,rimmel,[rimmel]
8,Milani,[Milani]
9,Curel/珂润,"[Curel, 珂润]"


In [5]:
CN_lst = []
EN_lst = []

for i in range(len(df)):
    
    if len(df['brand_name_split'][i]) == 2:
        EN = df['brand_name_split'][i][0]
        CN = df['brand_name_split'][i][1]
        
    if len(df['brand_name_split'][i]) == 1:
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['brand_name_split'][i][0])):
            CN = df['brand_name_split'][i][0]
            EN = ''
        else:
            CN = ''
            EN = df['brand_name_split'][i][0]
    
    CN_lst.append(CN)
    EN_lst.append(EN)
    
df['brandCN'] = CN_lst
df['brandEN'] = EN_lst

In [6]:
df.head()

,brand_name,brand_name_split,brandCN,brandEN
0,canmake/井田,"[canmake, 井田]",井田,canmake
1,SUQQU,[SUQQU],,SUQQU
2,three,[three],,three
3,Paula‘s Choice/宝拉珍选,"[Paula‘s Choice, 宝拉珍选]",宝拉珍选,Paula‘s Choice
4,Laroche Posay/理肤泉,"[Laroche Posay, 理肤泉]",理肤泉,Laroche Posay


In [7]:
len(df)

708

In [8]:
# 708 brands
df.to_csv("708brands.csv", encoding = "utf_8_sig")

# Impute Brand Name

In [11]:
df1 = pd.read_csv('708brands.csv', index_col = 0)
df2 = pd.read_csv('3432brands.csv', index_col = 0)

In [14]:
df1.head()
df2.head()

,brand_name,brand_name_split,brandCN,brandEN
0,canmake/井田,"['canmake', '井田']",井田,canmake
1,SUQQU,['SUQQU'],NaN,SUQQU
2,three,['three'],NaN,three
3,Paula‘s Choice/宝拉珍选,"['Paula‘s Choice', '宝拉珍选']",宝拉珍选,Paula‘s Choice
4,Laroche Posay/理肤泉,"['Laroche Posay', '理肤泉']",理肤泉,Laroche Posay


,brandCN,brandEN
0,奥蜜思,ORBIS
1,阿芙,AFU
2,伊蒂之屋,ETUDE HOUSE
3,艾诗缇,ASTALIFT
4,嫒碧知,episteme


In [15]:
for i in df1.index:
    if pd.isnull(df1.iloc[i]["brandCN"]):
        try:
            df1["brandCN"][i] = df2["brandCN"][df2[df2['brandEN'] == df1["brandEN"][i]].index.tolist()[0]]
        except:
            pass
        
for i in df1.index:
    if pd.isnull(df1.iloc[i]["brandEN"]):
        try:
            df1["brandEN"][i] = df2["brandEN"][df2[df2['brandCN'] == df1["brandCN"][i]].index.tolist()[0]]
        except:
            pass

In [16]:
df1.to_csv('filter708brands.csv')

# Brand Weibo Account Scrapy

In [18]:
brands = pd.read_csv('filter708brands.csv', keep_default_na = False)
brands = brands.loc[:, ~brands.columns.str.contains('^Unnamed')]

brands.head()
len(brands)

,brand_name,brand_name_split,brandCN,brandEN
0,canmake/井田,"['canmake', '井田']",井田,canmake
1,SUQQU,['SUQQU'],,SUQQU
2,three,['three'],,three
3,Paula‘s Choice/宝拉珍选,"['Paula‘s Choice', '宝拉珍选']",宝拉珍选,Paula‘s Choice
4,Laroche Posay/理肤泉,"['Laroche Posay', '理肤泉']",理肤泉,Laroche Posay


708

In [54]:
dflst = {}

for i in range(len(brands)//100+1):
    try:
        dflst[len(dflst)] = brands.iloc[i*100:(i+1)*100]
    except:
        dflst[len(dflst)] = brands.iloc[i*100:]

In [55]:
df1 = dflst[0]
df1 = df1.reset_index(drop = True)
df2 = dflst[1]
df2 = df2.reset_index(drop = True)
df3 = dflst[2]
df3 = df3.reset_index(drop = True)
df4 = dflst[3]
df4 = df4.reset_index(drop = True)
df5 = dflst[4]
df5 = df5.reset_index(drop = True)
df6 = dflst[5]
df6 = df6.reset_index(drop = True)
df7 = dflst[6]
df7 = df7.reset_index(drop = True)

lst1 = df_to_dict(df1)
lst2 = df_to_dict(df2)
lst3 = df_to_dict(df3)
lst4 = df_to_dict(df4)
lst5 = df_to_dict(df5)
lst6 = df_to_dict(df6)
lst7 = df_to_dict(df7)

In [56]:
def df_to_dict(brandsdf):
    brandlst = {}
    for i in range(len(brandsdf)):
        brandlst[len(brandlst)] = [brandsdf['brandCN'][i], brandsdf['brandEN'][i].upper()]
    return brandlst

In [38]:
def remove_special_chars(text):
    text = re.sub(r'[^a-zA-Z0-9\u4e00-\u9fff]' , '', text)
    return text

In [39]:
def get_brands(username, password, brandlist):
    
    driver = webdriver.Firefox()
    
    data = {}
    
    # try CN
    tdbrands = {}
    
    for i in tqdm(range(len(brandlist))):
        bdnames = brandlist[i]
        key_word = remove_special_chars(bdnames[0])
        if bdnames[0] == '':
            tdbrands[len(tdbrands)] = {'CN':bdnames[0], 'EN':bdnames[1]}
            continue
            
        
        driver.get('https://s.weibo.com/')
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/a[2]').click()
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').clear()
        
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').send_keys(key_word)
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/button').click()
        
        
        driver.find_element_by_link_text('找人').click()
        

        try:
            table = driver.find_element_by_xpath("//i[@class='icon-vip icon-vip-b']/../../..")
            text = table.find_element_by_xpath('div/a[1]').text
            text = remove_special_chars(text)
            if  ((key_word not in text) and (text not in key_word)):
                tdbrands[len(tdbrands)] = {'CN':bdnames[0], 'EN':bdnames[1]}
                continue

            name = table.find_element_by_xpath('div/a[1]').text   
            gender = table.find_element_by_xpath('p[1]/i').get_attribute('class')
            gender = re.findall('icon-sex-(.*)', gender)[0]
            region = table.find_element_by_xpath('p[1]').text.split(' ')[0]

            if table.find_element_by_xpath('p[3]').text[:2] == '关注':
                uid = table.find_element_by_xpath('p[3]/span[1]/a').get_attribute('href')
                uid = re.findall('[0-9]+', uid)[0]
                tag = table.find_element_by_xpath('p[2]').text.split(' ')
                follows = table.find_element_by_xpath('p[3]/span/a').text
                fans = table.find_element_by_xpath('p[3]/span[2]/a').text
                posts = table.find_element_by_xpath('p[3]/span[3]/a').text     

            else:
                uid = table.find_element_by_xpath('p[2]/span[1]/a').get_attribute('href')
                uid = re.findall('[0-9]+', uid)[0]
                tag = []
                follows = table.find_element_by_xpath('p[2]/span/a').text
                fans = table.find_element_by_xpath('p[2]/span[2]/a').text
                posts = table.find_element_by_xpath('p[2]/span[3]/a').text

            data[len(data)] = {'name':name, 'uid':uid, 'gender':gender, 'region':region,
                               'tag':tag, 'follows':follows, 'fans':fans, 'posts':posts,'brandsCN':bdnames[0],'brandsEN':bdnames[1]}

        except:
            tdbrands[len(tdbrands)] = {'CN':bdnames[0], 'EN':bdnames[1]}
        
    
            
    # try EN
    rmvbrands = {}
    for i in tqdm(range(len(tdbrands))):
        
        driver.get('https://s.weibo.com/')
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[1]/a[2]').click()
        time.sleep(3)

        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').clear()
        
            
        topic = tdbrands[i]['EN']
        topic = remove_special_chars(topic).lower()
        
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/div/input').send_keys(topic)
        driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/div[2]/button').click()
        time.sleep(3)
        
        driver.find_element_by_link_text('找人').click()
        time.sleep(3)
        
        if tdbrands[i]['EN'] == '':
            rmvbrands[len(rmvbrands)] = {'CN':tdbrands[i]['CN'], 'EN':tdbrands[i]['EN']}
            continue
        
        try:
            table = driver.find_element_by_xpath("//i[@class='icon-vip icon-vip-b']/../../..")
            text = table.find_element_by_xpath('div/a[1]').text
            text = remove_special_chars(text)
            if (topic not in text.lower()) and (text.lower() not in topic):
                rmvbrands[len(rmvbrands)] = {'CN':tdbrands[i]['CN'], 'EN':tdbrands[i]['EN']}
                continue
                
            name = table.find_element_by_xpath('div/a[1]').text   
            gender = table.find_element_by_xpath('p[1]/i').get_attribute('class')
            gender = re.findall('icon-sex-(.*)', gender)[0]
            region = table.find_element_by_xpath('p[1]').text.split(' ')[0]
            
            if table.find_element_by_xpath('p[3]').text[:2] == '关注':
                uid = table.find_element_by_xpath('p[3]/span[1]/a').get_attribute('href')
                uid = re.findall('[0-9]+', uid)[0]
                tag = table.find_element_by_xpath('p[2]').text.split(' ')
                follows = table.find_element_by_xpath('p[3]/span/a').text
                fans = table.find_element_by_xpath('p[3]/span[2]/a').text
                posts = table.find_element_by_xpath('p[3]/span[3]/a').text     
            
            else:
                uid = table.find_element_by_xpath('p[2]/span[1]/a').get_attribute('href')
                uid = re.findall('[0-9]+', uid)[0]
                tag = []
                follows = table.find_element_by_xpath('p[2]/span/a').text
                fans = table.find_element_by_xpath('p[2]/span[2]/a').text
                posts = table.find_element_by_xpath('p[2]/span[3]/a').text
                
            data[len(data)] = {'name':name, 'uid':uid, 'gender':gender, 'region':region,
                               'tag':tag, 'follows':follows, 'fans':fans, 'posts':posts,'brandsCN':tdbrands[i]['CN'],'brandsEN':tdbrands[i]['EN']}
       
        except:
            rmvbrands[len(rmvbrands)] = {'CN':tdbrands[i]['CN'], 'EN':tdbrands[i]['EN']}
       

    return data, rmvbrands

In [40]:
def dict_to_df(data):
    
    df = pd.DataFrame(data).T
    cols = ['name', 'uid', 'gender', 'region', 'tag', 'follows', 'fans', 'posts','brandsCN','brandsEN']
    df = df.loc[:,cols]
    
    for i in range(len(df)):
        
        df['tag'][i] = ', '.join(df['tag'][i])
        
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['follows'][i])):
            df['follows'][i] = df['follows'][i][:-1] + '0000'
        else:
             df['follows'][i] = df['follows'][i]
        
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['fans'][i])):
            df['fans'][i] = df['fans'][i][:-1] + '0000'
        else:
             df['fans'][i] = df['fans'][i]
                
        if bool(re.search(u'[\u4e00-\u9fa5]+', df['posts'][i])):
            df['posts'][i] = df['posts'][i][:-1] + '0000'
        else:
             df['posts'][i] = df['posts'][i]
                
    return df

## List 1

In [57]:
data1, rmvbrands1 = get_brands('bengyouwm@022435@163.com', 'uz942976', lst1)


100%|██████████| 52/52 [14:23<00:00, 17.68s/it]

In [47]:
data1 = dict_to_df(data1)
pd.DataFrame.from_dict(data1, orient = 'index').to_csv('data_0-100.csv', encoding = 'utf_8_sig')

,name,uid,gender,region,tag,follows,fans,posts,brandsCN,brandsEN
0,井田志Gridnote,5619422243,male,北京,原本一生（北京）商业有限公司,363,3488,494,井田,CANMAKE
1,宝拉珍选PaulasChoice,2040921197,female,上海,美国宝拉珍选化妆品官方微博,272,260000,7289,宝拉珍选,PAULA‘S CHOICE
2,理肤泉larocheposay,2051652844,female,海外,"理肤泉larocheposay, 官方微博",68,200000,6889,理肤泉,LAROCHE POSAY
3,SUQQU_official,6079473403,female,海外,"e'quipe,LTD",217,490000,786,,SUQQU
4,THREE,6320994446,female,海外,株式会社ACRO,14,80000,625,,THREE


In [ ]:
pd.DataFrame(rmvbrands1).T.to_csv('rmv_data_0-100.csv', encoding = 'utf_8_sig')

## List 2

In [ ]:
data2, rmvbrands2 = get_brands('bengyouwm@022435@163.com', 'uz942976', lst2)

In [ ]:
data2 = dict_to_df(data2)
pd.DataFrame.from_dict(data2, orient = 'index').to_csv('data_100-200.csv', encoding = 'utf_8_sig')

In [ ]:
pd.DataFrame(rmvbrands2).T.to_csv('rmv_data_100-200.csv', encoding = 'utf_8_sig')

## List 3

In [ ]:
data3, rmvbrands3 = get_brands('bengyouwm@022435@163.com', 'uz942976', lst3)

In [ ]:
data3 = dict_to_df(data3)
pd.DataFrame.from_dict(data3, orient = 'index').to_csv('data_200-300.csv', encoding = 'utf_8_sig')

In [ ]:
pd.DataFrame(rmvbrands3).T.to_csv('rmv_data_200-300.csv', encoding = 'utf_8_sig')

## List 4

In [ ]:
data4, rmvbrands4 = get_brands('bengyouwm@022435@163.com', 'uz942976', lst4)

In [ ]:
data4 = dict_to_df(data4)
pd.DataFrame.from_dict(data4, orient = 'index').to_csv('data_300-400.csv', encoding = 'utf_8_sig')

In [ ]:
pd.DataFrame(rmvbrands4).T.to_csv('rmv_data_300-400.csv', encoding = 'utf_8_sig')

## List 5

In [ ]:
data5, rmvbrands5 = get_brands('bengyouwm@022435@163.com', 'uz942976', lst5)

In [ ]:
data5 = dict_to_df(data5)
pd.DataFrame.from_dict(data5, orient = 'index').to_csv('data_400-500.csv', encoding = 'utf_8_sig')

In [ ]:
pd.DataFrame(rmvbrands5).T.to_csv('rmv_data_400-500.csv', encoding = 'utf_8_sig')

## List 6

In [ ]:
data6, rmvbrands6 = get_brands('bengyouwm@022435@163.com', 'uz942976', lst6)

In [ ]:
data6 = dict_to_df(data6)
pd.DataFrame.from_dict(data6, orient = 'index').to_csv('data_500-600.csv', encoding = 'utf_8_sig')

In [ ]:
pd.DataFrame(rmvbrands6).T.to_csv('rmv_data_500-600.csv', encoding = 'utf_8_sig')

## List 7

In [ ]:
data7, rmvbrands7 = get_brands('bengyouwm@022435@163.com', 'uz942976', lst7)

In [ ]:
data7 = dict_to_df(data7)
pd.DataFrame.from_dict(data7, orient = 'index').to_csv('data_600-708.csv', encoding = 'utf_8_sig')

In [ ]:
pd.DataFrame(rmvbrands7).T.to_csv('rmv_data_600-708.csv', encoding = 'utf_8_sig')